In [63]:
#importing libararies
import pandas as pd
import numpy as np
import os
from __future__ import print_function
from mailmerge import MailMerge
from datetime import date
import docx
import import_ipynb
import glob
import tkinter
from tkinter import filedialog
import warnings
warnings.filterwarnings("ignore")


def format_sheet(file_path:str,template_path:str,sheet_num_template=0,run=0):
    """
    file_path: the file you want to design, must bx xlsx
    template_path: the teplate of the design, must bx xlsx
        """
    # format sheet
    xlsx=pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    for i in range(len(xlsx.book.worksheets)):
        new_sheet=xlsx.book.worksheets[run]
        template=pd.ExcelWriter(template_path,engine='openpyxl', mode='a')
        default_sheet=template.book.worksheets[sheet_num_template]
        from copy import copy
        new_sheet.sheet_view.rightToLeft=True
        for row in default_sheet.rows:
            for cell in row:
                new_cell=new_sheet.cell(row=cell.row, column=cell.col_idx)
                if cell.has_style and new_cell.value != None:
                    new_cell.font=copy(cell.font)
                    new_cell.border=copy(cell.border)
                    new_cell.fill=copy(cell.fill)
                    new_cell.number_format=copy(cell.number_format)
                    new_cell.protection=copy(cell.protection)
                    new_cell.alignment=copy(cell.alignment)
        from openpyxl.utils import get_column_letter
        for i in range(default_sheet.max_column):
            new_sheet.column_dimensions[get_column_letter(i + 1)].width=default_sheet.column_dimensions[
                get_column_letter(i + 1)].width
    
    xlsx.save()
def change_names_and_order(file_path:str,df:pd.DataFrame):
    """
    name of columns in names must be "old_name", "new_name"
        """
    match_names=pd.read_excel(file_path)
    match_names.index=match_names.old_name
    dict_names=match_names.to_dict()['new_name']
    ls_names=match_names['new_name'].tolist()
    df.rename(columns=dict_names, inplace=True)
    df=df[ls_names]
    return df

In [84]:

tkinter.Tk().withdraw() # prevents an empty tkinter window from appearing

transction_folder_path = filedialog.askdirectory()
permanant_folder_path = filedialog.askdirectory()

In [228]:
#reading the check file
filepath_check = transction_folder_path+"\*transfers*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    check_sent = pd.read_excel(textfile,sheet_name="Transfers",skiprows=1)
    


#reading the risk country file
filepath_check = permanant_folder_path+"\*מדינות*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    rc = pd.read_excel(textfile)

#reading the rates file
filepath_check = permanant_folder_path+"\*שערים*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    rates = pd.read_excel(textfile ,sheet_name="CUR CONV",usecols="F:G",skiprows=1)
    

#reading the report number generator file
filepath_check = transction_folder_path+"\*מחולל*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    report_num = pd.read_excel(textfile)
    
#reading the report content file
filepath_check = transction_folder_path+"\*content*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    content = pd.read_excel(textfile)  

filepath_check = transction_folder_path+"\*Branch*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    Branches = pd.read_excel(textfile)  

In [229]:
check_sent = check_sent.rename(columns = {'№ перевода':'Order No','Операция':'transction type','Текущий статус перевода':'transction status','Дата отправки, МСК':'TX Date','ФИО':'Sender','Дата рождения':'Cust DOB','Паспорт':'Cust ID NO','Страна выдачи паспорта':'ID Issued BY Country','ФИО.1':'Receiver','Страна выдачи паспорта.1':'Ben Country','Страна назначения':'dest country','Валюта приема':'Currency','Сумма платежа ':'Amount','Агент отправитель':'Agent'})
check_sent = check_sent[((check_sent['transction type'] == 'Отправка') & ((check_sent['transction status'] == 'Запрошен на выдачу') | (check_sent['transction status'] == 'Выдан') | (check_sent['transction status'] =='Готов к выдаче') ))]

In [230]:
#getting the data ready
#naming rates columns
rates.columns=["curr","rate"]

#adding the rates value
check_sent= check_sent.merge(rates, how = 'left', left_on = 'Currency', right_on = 'curr')
check_sent['nis sum'] = check_sent['rate'] * check_sent['Amount']

#lower the country cases:
check_sent['ID Issued BY Country'] = check_sent['ID Issued BY Country'].str.lower()
check_sent['Ben Country'] = check_sent['Ben Country'].str.lower()
check_sent['dest country'] = check_sent['dest country'].str.lower()
rc['RUSSIAN'] = rc['RUSSIAN'].str.lower()
check_sent = check_sent.fillna('')

#risk check
#check_sent = check.merge(rc[['English short name (using title case)','סיכון/ללא דיווח רגיל']] , left_on = 'Source country', right_on = 'English short name (using title case)',how = 'left',suffixes = ('','_source'))
check_sent = check_sent.merge(rc[['RUSSIAN','סיכון/ללא דיווח רגיל']] , left_on = 'Ben Country', right_on = 'RUSSIAN',how = 'left',suffixes = ('','_ben'))
check_sent = check_sent.merge(rc[['RUSSIAN','סיכון/ללא דיווח רגיל']] , left_on = 'ID Issued BY Country', right_on = 'RUSSIAN',how = 'left',suffixes = ('','_issuance'))
check_sent = check_sent.merge(rc[['RUSSIAN','סיכון/ללא דיווח רגיל']] , left_on = 'dest country', right_on = 'RUSSIAN',how = 'left',suffixes = ('','_dest'))


In [231]:
#date coolums as dat and time and a columns with the month number
check_sent['TX Date']= pd.to_datetime(check_sent['TX Date'],dayfirst=True)
check_sent['month'] = pd.DatetimeIndex(check_sent['TX Date']).month
check_sent['sender_month'] = (check_sent['month']).astype(str) + (check_sent['Sender'])
check_sent['Receiver_month'] = (check_sent['month']).astype(str)  +(check_sent['Receiver'])

In [232]:
#check risk 
check_risk_sent = check_sent[((check_sent['סיכון/ללא דיווח רגיל'].notna()) | (check_sent['סיכון/ללא דיווח רגיל_issuance'].notna()) | (check_sent['סיכון/ללא דיווח רגיל_dest'].notna()))]

#building the monitoring df with sum per client, deal distinct count and under 50k deal count
sent_monitoring = check_sent.groupby(['Sender','month']).agg({'nis sum':'sum','Order No':'nunique'})
sent_monitoring = sent_monitoring.reset_index()
sent_monitoring_excel = sent_monitoring

In [233]:
#sorting the values by sum
sent_monitoring = sent_monitoring.sort_values('nis sum',ascending = False)
sent_monitoring_excel = sent_monitoring
#filter only client over 47K
sent_monitoring = sent_monitoring[sent_monitoring['nis sum'] >= 47000]

#adding a column named status based on unique count of deal number
sent_monitoring = sent_monitoring[((sent_monitoring['Order No'] > 1) & (sent_monitoring['nis sum'] >= 50000))|(sent_monitoring['nis sum'] < 50000)]

#marking dirdos transction as d in a new column named dirdos
sent_monitoring['type'] = np.where((sent_monitoring['nis sum']>= 47000) & (sent_monitoring['nis sum']<50000) , "d",'p')
sent_monitoring=sent_monitoring.reset_index()

#out bound risk
#check_sent_risk = check_sent[(check_sent['סיכון/ללא דיווח רגיל'].notna()) | (check_sent['סיכון/ללא דיווח רגיל_dest'].notna()) | (check_sent['סיכון/ללא דיווח רגיל_issuance'].notna())]

#outbound tisk monitorng df
sent_risk_monitoring = check_risk_sent.groupby(['Sender','month']).agg({'nis sum':'sum','Order No':'nunique'})

#sorting the values by sum
sent_risk_monitoring = sent_risk_monitoring.sort_values('nis sum',ascending = False)

#filtering to report tr
sent_risk_monitoring = sent_risk_monitoring[sent_risk_monitoring['nis sum'] >= 4700]
sent_risk_monitoring = sent_risk_monitoring[((sent_risk_monitoring['Order No'] > 1) & (sent_risk_monitoring['nis sum'] >= 5000)) | (sent_risk_monitoring['nis sum'] < 5000)]


#adding the type of the report pr-pitzul dr-dirdos (risk)
sent_risk_monitoring['type'] =  np.where((sent_risk_monitoring['nis sum'] >= 5000), "pr" , "dr")
sent_risk_monitoring = sent_risk_monitoring.reset_index()

try:
    report = sent_risk_monitoring[['type','Sender','month']].merge(sent_monitoring[['type','Sender','month']], on = 'Sender',how = 'outer',suffixes = ('_risk',''))
except KeyError:
    report = sent_risk_monitoring[['type','Sender','month']]
    print('no no risk reports')
    
#building the monitoring df with sum per client and deal distinct count
check_sent_ow = check_sent.groupby(['Receiver','month']).agg({'nis sum':'sum','Order No':'nunique','Sender':'nunique'})

#sorting the values by sum
check_sent_ow = check_sent_ow.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
check_sent_ow = check_sent_ow[(check_sent_ow['Order No'] > 1) & (check_sent_ow['Sender'] > 1) & (check_sent_ow['nis sum'] >= 50000)]
check_sent_ow = check_sent_ow.reset_index()

#creating a list of the shared owner report clients name
check_sent_ow = check_sent_ow.merge(check_sent[['Sender','Receiver']], on = 'Receiver', how = 'left',suffixes = ('_count','') )
check_sent_ow = check_sent_ow.merge(report, on ='Sender' , how = 'left', suffixes = ('','_report'))


try:
    check_sent_ow = check_sent_ow[ (check_sent_ow['type'].isna()) & (check_sent_ow['type_risk'].isna())]
except KeyError:
    check_sent_ow =  check_sent_ow[(check_sent_ow['type_risk'].isna())]
        
check_sent_ow['type_m'] = 'm'
check_sent_ow['Receiver_month'] = (check_sent_ow['month']).astype(str) + (check_sent_ow['Receiver'])
check_sent_ow_dup = check_sent_ow
check_sent_ow = check_sent_ow.drop_duplicates('Receiver')
try:
    report = check_sent_ow[['Sender','type_m','month']].merge(report, on = 'Sender', how = 'outer')
except KeyError:
    report = report
    print('no shared owner reports')    
    
#building the monitoring df with sum per client and deal distinct count risk
check_sent_ow_risk = check_risk_sent.groupby(['Receiver','month']).agg({'nis sum':'sum','Order No':'nunique','Sender':'nunique'})

#sorting the values by sum
check_sent_ow_risk = check_sent_ow_risk.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
check_sent_ow_risk = check_sent_ow_risk[(check_sent_ow_risk['Order No'] > 1) & (check_sent_ow_risk['Sender'] > 1) & (check_sent_ow_risk['nis sum'] >= 5000)]
check_sent_ow_risk = check_sent_ow_risk.reset_index()

#creating a list of the shared owner report clients name
check_sent_ow_risk = check_sent_ow_risk.merge(check_risk_sent[['Sender','Receiver']], on = 'Receiver', how = 'left',suffixes = ('_count','') )
check_sent_ow_risk = check_sent_ow_risk.merge(report, on ='Sender' , how = 'left')

try:
    check_sent_ow_risk = check_sent_ow_risk[(check_sent_ow_risk['type'].isna()) & (check_sent_ow_risk['type_risk'].isna()) & (check_sent_ow_risk['type_m'].isna())]
except KeyError:
    check_sent_ow_risk =  check_sent_ow_risk
    print('error')

check_sent_ow_risk['type_m_risk'] = 'mr'
check_sent_ow_risk['Receiver_month'] = (check_sent_ow_risk['month']).astype(str) + (check_sent_ow_risk['Receiver'])
check_sent_ow_risk_dup = check_sent_ow_risk
check_sent_ow_risk = check_sent_ow_risk.drop_duplicates('Receiver')

#merging the risk ow with regular report
try:
    report = check_sent_ow_risk[['Sender','type_m_risk','month']].merge(report, on = 'Sender', how = 'outer')
except KeyError:
    print('error')
    report = report

In [234]:
#creating the details to report inbound and outbound
#outbound
report = report.merge(check_sent[['Sender','Ben Country','Receiver','Cust DOB','dest country','Cust ID NO','ID Issued BY Country','sender_month']],on = 'Sender',how = 'left')
report = report.drop_duplicates('Sender')
report = report.fillna('')
report['final type'] = report['type']+report['type_risk']+report['type_m'] + report['type_m_risk']

full_report = report.fillna('')


In [235]:
#importing the report number needed
report_id = (report_num.iloc[1,5] + 1).astype(int)
full_report['report_id'] = report_id + range(len(full_report.index))
full_report['report_name'] = report_num.iloc[3,1].rsplit('-',1)[0]


In [236]:

#checking the risk type
risk_country4report = full_report.merge(rc[['RUSSIAN','עברית','סיכון/ללא דיווח רגיל']], left_on = 'ID Issued BY Country', right_on ='RUSSIAN', how = 'left')
risk_country4report = risk_country4report.merge(rc[['RUSSIAN','עברית','סיכון/ללא דיווח רגיל']], left_on = 'Ben Country', right_on ='RUSSIAN', how = 'left',suffixes = ('','_ben'))
risk_country4report = risk_country4report.merge(rc[['RUSSIAN','עברית','סיכון/ללא דיווח רגיל']], left_on = 'dest country', right_on ='RUSSIAN', how = 'left',suffixes = ('','_dest'))



In [237]:
risk_country4report['risk_full_content'] = ''
#if statment for risk country for report
for i in risk_country4report.index:
    if ((risk_country4report['סיכון/ללא דיווח רגיל'][i] == 'סיכון') | (risk_country4report['סיכון/ללא דיווח רגיל'][i] == 'סיכון ללא דיווח רגיל')):
        risk_country4report['risk_full_content'][i] = 'הלקוח תושב ' +  risk_country4report['עברית'][i] + ', מדינה המוגדרת בסיכון גבוה'
    elif ((risk_country4report['סיכון/ללא דיווח רגיל_dest'][i] == 'סיכון') | (risk_country4report['סיכון/ללא דיווח רגיל_dest'][i] == 'סיכון ללא דיווח רגיל')):
        risk_country4report['risk_full_content'][i] = 'הכספים נשלחו ל' +  risk_country4report['עברית_dest'][i] + ', מדינה המוגדרת בסיכון גבוה'
    elif ((risk_country4report['סיכון/ללא דיווח רגיל_ben'][i] == 'סיכון') | (risk_country4report['סיכון/ללא דיווח רגיל_ben'][i] == 'סיכון ללא דיווח רגיל')):
        risk_country4report['risk_full_content'][i] = 'המוטב תושב ' +  risk_country4report['עברית_ben'][i] + ', מדינה המוגדרת בסיכון גבוה' 
    else :
        risk_country4report['risk_full_content'][i] = ''


In [238]:

risk_country4report = risk_country4report.merge(content, left_on = 'final type' , right_on = 'type', how = 'left')

final_client2report = risk_country4report[['risk_full_content','Content','Title','עברית_dest','עברית','report_name','report_id','final type','Cust ID NO','Cust DOB','Sender','month' ]]




In [239]:
final_client2report = final_client2report.merge(check_sent[['Sender','Agent']], on = 'Sender', how = 'left')
final_client2report = final_client2report.merge(Branches[['Код отделения в файле ','Отделение','סעיף 5']],left_on = 'Agent',right_on = 'Код отделения в файле ',how = 'left')
final_client2report = final_client2report.rename(columns = {'Отделение':'final agent'})


In [223]:
final_client2report = final_client2report.fillna('')
final_client2report_dup = final_client2report.drop_duplicates(subset = ['Sender','month'])
final_client2report_dup['month'] = final_client2report_dup['month'].astype(int)
final_client2report_dup['sender_month'] = (final_client2report_dup['month']).astype(str) + (final_client2report_dup['Sender'])
final_client2report = final_client2report.drop_duplicates('Sender')
final_client2report.to_excel("monitoring.xlsx")

ValueError: invalid literal for int() with base 10: ''

In [ ]:
#creating an excel filies for monitoring the results
monitoring_name_excel = sent_monitoring_excel.merge(final_client2report[['Sender','final type']], on = 'Sender', how = 'left')
monitoring_name_excel['status'] = ''
for i in monitoring_name_excel.index:
        
    if monitoring_name_excel['nis sum'][i] <=47000 :
        monitoring_name_excel['status'][i] = 'סך עסקאות נמוך מרף הדיווח'

    elif monitoring_name_excel['Order No'][i] <= 1 :
        monitoring_name_excel['status'][i] = 'הועבר דיווח רגיל'
    else :
        monitoring_name_excel['status'][i] = monitoring_name_excel['final type'][i]
        
monitoring_name_excel = monitoring_name_excel.sort_values('nis sum', ascending = False)
monitoring_name_excel['final type'] = monitoring_name_excel['final type'].fillna(monitoring_name_excel['status'])
monitoring_name_excel = monitoring_name_excel.rename(columns= {'final type':'final status'})
monitoring_name_excel['final status'] = monitoring_name_excel['final status'].replace({'pr':'פיצול בסיכון','p':'פיצול','dr':'דירדוס בסיכון','d':'דירדוס','m':'מושך משותף','mr':'מושך משותף בסיכון'})

#xlsx = pd.ExcelWriter('monitoring.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay')
xlsx=pd.ExcelWriter('monitoring.xlsx', engine='openpyxl', mode='a',if_sheet_exists='overlay')
monitoring_name_excel[['Sender','nis sum','Order No','final status','month']].to_excel(xlsx, sheet_name='ניטור',index=False)
xlsx.save()

In [ ]:
#adding the hebrew name for destnation country
check_sent = check_sent.merge(rc[['RUSSIAN','עברית']], left_on = 'dest country', right_on = 'RUSSIAN' , how = 'left')

In [ ]:

filepath_check = transction_folder_path+"\*טמפלייט*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    template = textfile
document = MailMerge(template)
print(document.get_merge_fields())



for i in final_client2report.index:
     #template1 = 'test1.docx'
    document = MailMerge(template)
    document.merge(
        Sender = str(final_client2report['Sender'][i]),
        
        Cust_DOB = str(final_client2report['Cust DOB'][i]),
        Title = str(final_client2report['Title'][i]),
        report_id = str(final_client2report['report_id'][i]),
        Content = str(final_client2report['Content'][i]),
        risk_full_content = str(final_client2report['risk_full_content'][i]),
        עברית = str(final_client2report['עברית'][i]),
        Cust_ID_NO = str(final_client2report['Cust ID NO'][i]),
        final_agent = str(final_client2report['final agent'][i]),
        סעיף_5 = str(final_client2report['סעיף 5'][i])
        
        )
    

    output =  str(final_client2report['report_name'][i]) + "-" + str(final_client2report['report_id'][i]) + '.docx'
    document.write(output)

#filtering outbound reports
sent_report = final_client2report_dup[(final_client2report_dup['final type'] == 'p') |(final_client2report_dup['final type'] == 'd') |(final_client2report_dup['final type'] == 'dr') |(final_client2report_dup['final type'] == 'pr') | (final_client2report_dup['final type'] == 'ppr')]

#creating a data frames with only need to report transctions
sent_report_tr = check_sent[check_sent['sender_month'].isin(sent_report['sender_month'])]


#leaving only the imporatant columns
sent_report_tr = sent_report_tr[['Order No','TX Date','עברית','Sender','Receiver','Amount','Currency','Cust ID NO','nis sum']]

#creating an excel shhet with the ountbound reports
i=0
for_word_table=[]
xlsx=pd.ExcelWriter("monitoring.xlsx", engine='openpyxl', mode='a',if_sheet_exists='overlay')
#round the sum in ils
sent_report_tr["nis sum"]=sent_report_tr["nis sum"].round()

#creating a unique list of the sender names
ls_customer=sent_report_tr["Sender"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer:
    df=sent_report_tr[sent_report_tr["Sender"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["Sender", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=i,index=False)
    for_word_table.append(orderd)
    i=i+len(orderd)+2
            
    xlsx.save()
    
#indexing the outbound_report data frame
info2r = sent_report.set_index('Sender')
info2r_nona = info2r[info2r.index.notnull()]

for title in ls_customer:
    
    data = sent_report_tr[sent_report_tr['Sender'] == title].round()
    table = [["סהכ",data["nis sum"].sum().round(),data['Amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["Sender", "nis sum",'Amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(' ')
    
    orderd=change_names_and_order(transction_folder_path+r"/change_names.xlsx",orderd)
    
    orderd['תאריך פעולה'] = orderd['תאריך פעולה'].astype(str)
    
    doc = docx.Document(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])
                        
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
    
    doc.save(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")

#filtering only shared owner reports
sent_report_ow = final_client2report_dup[(final_client2report_dup['final type'] == 'm') |(final_client2report_dup['final type'] == 'mr')]

#concatacating the two kinds of outbound shared owner transction dataframes
sent_tr_ow = pd.concat([check_sent_ow_risk_dup,check_sent_ow_dup])

#filtering only need to report transctions
sent_report_tr_ow = check_sent[check_sent['Receiver_month'].isin(sent_tr_ow['Receiver_month'])]

#leaving only the imporatant columns
sent_report_tr_ow = sent_report_tr_ow[['Order No','TX Date','עברית','Sender','Receiver','Amount','Currency','Cust ID NO','nis sum']]

#creating a new list
j=0
for_word_table_ow=[]

#wrting an excel file containing the report transctions for each customer
xlsx=pd.ExcelWriter("monitoring.xlsx", engine='openpyxl', mode='a',if_sheet_exists='overlay')

#round the sum in ils
sent_report_tr_ow["nis sum"]=sent_report_tr_ow["nis sum"].round()

#creating a unique list of the sender names
ls_customer_ow=sent_report_tr_ow["Receiver"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer_ow:
    df=sent_report_tr_ow[sent_report_tr_ow["Receiver"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["Sender", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='ow_Reports',startrow=j,index=False)
    for_word_table_ow.append(orderd)
    j=j+len(orderd)+2
            
    xlsx.save()
    
#creating a list containing the client name and owner name
info2r_ow = sent_report_ow[['report_name','report_id','Sender']].merge(sent_report_tr_ow[['Sender','Receiver']], on = 'Sender', how = 'left')
info2r_ow = info2r_ow.drop_duplicates('Receiver')
info2r_ow = info2r_ow.set_index('Receiver')


for title in ls_customer_ow:    
    data = sent_report_tr_ow[sent_report_tr_ow['Receiver'] == title]
    table = [["סהכ",data["nis sum"].sum().round(),data['Amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["Receiver", "nis sum",'Amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(" ")
    orderd=change_names_and_order(transction_folder_path+r"/change_names.xlsx",orderd)
    
    doc = docx.Document(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])                   
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
   
    doc.save(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")
    
    
